In [23]:
from pathlib import Path
from json import loads

import rosbag
import rospy
import numpy as np
from chabo_common.utils.math_utils import *

In [68]:
cones = []
poses = []

last_pose_t = 0
last_cone_t = 0

with rosbag.Bag('/mnt/c/Users/papal/Downloads/fss_4_laps.bag', 'r') as bag:
    
    for topic, msg, t in bag.read_messages(["/slam/cones", "/slam/pose"]):
        # print(topic, t)
        if 'cones'  in topic and last_pose_t > last_cone_t:
            last_cone_t = t.to_sec()
            cones.append(msg)
        elif 'pose' in topic:
            last_pose_t = t.to_sec()
            poses.append(msg)

len(poses), len(cones)

(1384, 1381)

In [69]:
from fsd_path_planning import ConeTypes

def convert_cone_observation(cone_observation) -> np.ndarray:
    out = [[] for _ in ConeTypes]
    for cone_type in ConeTypes:
        
        cones_of_type = getattr(cone_observation, cone_type.name.lower())
        for c in cones_of_type:
            out[cone_type].append([c.x, c.y])
        
    return out


def convert_pose(pose) -> np.ndarray:
    pos = [pose.pose.position.x, pose.pose.position.y]
    angle = quaternion_to_euler_angles(np.array([pose.pose.orientation.x, pose.pose.orientation.y, pose.pose.orientation.z, pose.pose.orientation.w]))[2]
    direction = unit_2d_vector_from_angle(angle).tolist()

    return pos, direction



    

In [70]:
data = []


for p, c in zip(poses, cones):
    pos, dire = convert_pose(p)
    con = convert_cone_observation(c.cone_observation)
    d = {
        "car_position": pos,
        "car_direction": dire,
        "slam_cones": con
    }
    data.append(d)

len(data)

1381

In [71]:
[np.array(x).reshape(-1,2) for x in data[-1]['slam_cones']]

[array([], shape=(0, 2), dtype=float64),
 array([[  1.24879456,   8.75      ],
        [  2.18629456,  18.1875    ],
        [  1.24879456,  12.375     ],
        [  3.12379456,  19.6875    ],
        [  1.43629456,  16.3125    ],
        [  5.56129456,  20.8125    ],
        [ 14.74879456,  24.9375    ],
        [ 18.68629456,  25.6875    ],
        [ 21.31129456,  25.6875    ],
        [ 11.74879456,  23.625     ],
        [ 23.81129456,  25.875     ],
        [ 26.62379456,  26.0625    ],
        [ 27.89708838,  22.13631838],
        [ 30.18629456,  26.25      ],
        [ 31.12379456,  22.3125    ],
        [ 33.37379456,  27.        ],
        [ 33.56129456,  22.6875    ],
        [ 36.74879456,  23.0625    ],
        [ 36.93629456,  28.3125    ],
        [ 39.56129456,  29.4375    ],
        [ 39.93629456,  23.4375    ],
        [ 41.06129456,  31.125     ],
        [ 42.37379456,  24.1875    ],
        [ 42.74879456,  31.875     ],
        [ 44.43629456,  24.9375    ],
        [

In [72]:
import json

p = Path('/home/papalotis/repos/fsd-path-planning/fsd_path_planning/demo/fss_19_4_laps.json')
p.write_text(json.dumps(data))

6529579